In [ ]:
"""Calculating P&L on strategies

Demonstration of how to use sub-holding keys and output transactions to track P&L on different strategies.

Attributes
----------
output transactions
transactions
properties
sub-holding keys
derived portfolios
cocoon - seed_data
"""

# Calculating P&L on strategies

This notebook demonstrates how you can use LUSID's [Sub-holding Keys](https://support.finbourne.com/what-are-subholding-keys) (or SHKs) and [BuildTransactions](https://www.lusid.com/docs/api/#operation/BuildTransactions) endpoint to track P&L on different strategies. In this notebook we will consider a simple example of a portfolio which executes multiple trades on <i>Tesco PLC</i> across two different strategies throughout 2019:

* The <b>quant_strategy</b> which makes trades based on a quantitative model
* The <b>fundamental_food_retail</b> strategy which makes trades based on fundamental research

At the end of the period, we use LUSID to calculate the P&L per strategy. We also use LUSID's [derived portfolios](https://support.finbourne.com/what-is-a-derived-portfolio) to calculate P&L using two different accounting methods of `FIFO` and `AverageCost`.

### Setup LUSID

In [1]:
# Import LUSID
import lusid.models as models
from lusidjam import RefreshingToken
import lusid
import lusidtools.cocoon.cocoon as cocoon
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon_printer import format_transactions_response
from lusidtools.cocoon.transaction_type_upload import (
    create_transaction_type_configuration,
)

# Import Libraries
import pprint
import pytz
import pandas as pd
import numpy as np
import json
import requests
import os
import warnings
from datetime import datetime, timedelta, time

pd.set_option("display.max_columns", None)

# Configure notebook logging and warnings
import logging

logger = logging.getLogger()
logger.setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version: ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
LUSID API Version:  0.5.4411.0


Load a mapping file for DataFrame headers for the `build transaction` and `get holdings` response.

In [2]:
with open(r"config/build_transactions_mapping.json") as mappings_file:
    build_transactions_json_mapping = json.load(mappings_file)

with open(r"config/get_holdings_mapping.json") as mappings_file:
    get_holdings_json_mapping = json.load(mappings_file)

Define our transaction and derived portfolios API

In [3]:
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
derived_portfolio_api = api_factory.build(lusid.api.DerivedTransactionPortfoliosApi)

In [4]:
def get_build_transactions_df(portfolio_code):

    response = transaction_portfolios_api.build_transactions(
        scope=scope,
        code=portfolio_code,
        transaction_query_parameters=lusid.models.TransactionQueryParameters(
            start_date="2019-01-01", end_date="2019-12-31", query_mode="TradeDate"
        ),
        property_keys=["Instrument/default/Name"],
    )

    build_transactions_df = lusid_response_to_data_frame(
        response,
        rename_properties=True,
        column_name_mapping=build_transactions_json_mapping,
    )

    build_transactions_df.rename(
        columns={
            "realised_gain_loss.0.realised_trade_ccy.amount": "PnL",
            f"strategy({scope}-Properties)": "Strategy",
        },
        inplace=True,
    )

    return build_transactions_df

### 1) Create a scope and load a CSV file of Tesco PLC trades

* There are 8 Tesco PLC trades in total over 2019 (4 per strategy).
* The trades are executed at various prices.
* There are Buys and Sells in both strategies.

In [5]:
# Create a new scope

scope = "strategy-pnl-notebook"
portfolio_code = "EQUITY-STRATEGY" 

In [6]:
# Load a file of equity transactions

transactions_file = r"data/pnl/strategy_pnl.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code
transactions_df

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions
0,EQUITY-STRATEGY,Equity retail multi-strategy,GBP,GB0008847096,SEDOL7,equity,EQ_1240,Tesco PLC,trd_001,Buy,01/01/2019,01/01/2019,1000000,8,8000000,GBP,quant_strategy,NaN
1,EQUITY-STRATEGY,Equity retail multi-strategy,GBP,GB0008847096,SEDOL7,equity,EQ_1240,Tesco PLC,trd_002,Buy,01/01/2019,01/01/2019,1000000,8,8000000,GBP,fundamental_food_retail,NaN
2,EQUITY-STRATEGY,Equity retail multi-strategy,GBP,GB0008847096,SEDOL7,equity,EQ_1240,Tesco PLC,trd_003,Sell,01/02/2019,01/02/2019,400000,10,4000000,GBP,quant_strategy,NaN
3,EQUITY-STRATEGY,Equity retail multi-strategy,GBP,GB0008847096,SEDOL7,equity,EQ_1240,Tesco PLC,trd_004,Sell,01/03/2019,01/03/2019,400000,2,800000,GBP,fundamental_food_retail,NaN
4,EQUITY-STRATEGY,Equity retail multi-strategy,GBP,GB0008847096,SEDOL7,equity,EQ_1240,Tesco PLC,trd_005,Buy,01/04/2019,01/04/2019,1000000,10,10000000,GBP,quant_strategy,NaN
5,EQUITY-STRATEGY,Equity retail multi-strategy,GBP,GB0008847096,SEDOL7,equity,EQ_1240,Tesco PLC,trd_006,Buy,01/05/2019,01/05/2019,1000000,12,12000000,GBP,fundamental_food_retail,NaN
6,EQUITY-STRATEGY,Equity retail multi-strategy,GBP,GB0008847096,SEDOL7,equity,EQ_1240,Tesco PLC,trd_007,Sell,01/06/2019,01/06/2019,300000,11,3300000,GBP,quant_strategy,NaN
7,EQUITY-STRATEGY,Equity retail multi-strategy,GBP,GB0008847096,SEDOL7,equity,EQ_1240,Tesco PLC,trd_008,Sell,01/07/2019,01/07/2019,300000,7,2100000,GBP,fundamental_food_retail,NaN
8,EQUITY-STRATEGY,Equity retail multi-strategy,GBP,GBP,GBP,cash,GBP,GBP Cash,cash_001,FundsIn,01/01/2019,01/01/2019,20000000,1,20000000,GBP,quant_strategy,GBP
9,EQUITY-STRATEGY,Equity retail multi-strategy,GBP,GBP,GBP,cash,GBP,GBP Cash,cash_002,FundsIn,01/01/2019,01/01/2019,20000000,1,20000000,GBP,fundamental_food_retail,GBP


### 2) Create a property called "strategy"

The <b>strategy</b> property will be used to create our `Sub-Holding Key` on the portfolio.

In [7]:
domain = "Transaction"
scope = scope
prop_code = "strategy"

try:
    api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
        create_property_definition_request=lusid.models.CreatePropertyDefinitionRequest(
            domain=domain,
            scope=scope,
            code=prop_code,
            value_required=None,
            display_name="The portfolio's investment strategy",
            data_type_id=lusid.ResourceId(scope="system", code="string"),
            life_time=None,
        )
    )

except lusid.exceptions.ApiException as e:
    print(json.loads(e.body)["title"])

Error creating Property Definition 'Transaction/strategy-pnl-notebook/strategy' because it already exists.


### 3) Load default transactions into a new scope

The portfolio is created and transaction history for 2019 is loaded. 

In [8]:
# Load portfolios, instruments, and transactions

seed_data_response = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_df,
    "DataFrame",
    sub_holding_keys=[f"Transaction/{scope}/strategy"],
)

### 4) Create two new derived portfolios with different accounting methods: FIFO and AverageCost

The derived portfolio inherits <u>all transactions and properties</u> from the parent portfolio. However on our new derived portfolio we change the accounting method to either <b>FIFO</b> or <b>Average Cost</b>. All other attributes and history are unchanged. This allows us to run a comparison between the two P&L methodologies.

In [9]:
for accounting_method in ["FirstInFirstOut", "AverageCost"]:

    derived_portfolio_code = portfolio_code + "_" + accounting_method

    try:
        derived_portfolio_api.create_derived_portfolio(
            scope=scope,
            create_derived_transaction_portfolio_request=lusid.models.CreateDerivedTransactionPortfolioRequest(
                code=derived_portfolio_code,
                display_name=f"{accounting_method}Accounting treatment for the fund",
                parent_portfolio_id=lusid.models.ResourceId(
                    scope=scope, code=portfolio_code
                ),
                accounting_method=accounting_method,
                created=datetime(year=2010, month=1, day=1, tzinfo=pytz.UTC).isoformat(),
            ),
        )

        print(f"The derived portfolio {derived_portfolio_code} has been created.")

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

Could not create a portfolio with id EQUITY-STRATEGY_FirstInFirstOut because it already exists in scope strategy-pnl-notebook.
Could not create a portfolio with id EQUITY-STRATEGY_AverageCost because it already exists in scope strategy-pnl-notebook.


### 5) Check P&L on the FirstInFirstOut (FIFO) portfolio using the Build Transactions method

Result:

* The <b>quant_strategy</b> performed better over 2019

In [10]:
fifo_portfolio = portfolio_code + "_" + "FirstInFirstOut"

build_transactions_df = get_build_transactions_df(fifo_portfolio)
build_transactions_df = build_transactions_df[
    build_transactions_df["LusidInstrumentId"] != "CCY_GBP"
]

build_transactions_df[["PnL", "InstrumentName", "Strategy"]].groupby(
    ["Strategy", "InstrumentName"]
).sum()

,,PnL
Strategy,InstrumentName,
fundamental_food_retail,Tesco PLC,-2700000.0
quant_strategy,Tesco PLC,1700000.0


### 6) Check P&L on the AverageCost portfolio using the Build Transactions method

Result:

* The <b>quant_strategy</b> performed better over 2019
* However the P&L is less favourable when using the AverageCost versus FIFO method

In [11]:
ac_portfolio = portfolio_code + "_" + "AverageCost"

build_transactions_df = get_build_transactions_df(ac_portfolio)
build_transactions_df = build_transactions_df[
    build_transactions_df["LusidInstrumentId"] != "CCY_GBP"
]

build_transactions_df[["PnL", "InstrumentName", "Strategy"]].groupby(
    ["Strategy", "InstrumentName"]
).sum()

,,PnL
Strategy,InstrumentName,
fundamental_food_retail,Tesco PLC,-3262500.0
quant_strategy,Tesco PLC,1137500.0
